In [2]:
#the project is completed on IBM Watson cloud, and data was exported to sql database on cloud,
#However due to extremely large dataset, I have only chosen 3 data files for training as the amount of data was extremly large

In [21]:
import ibm_db
import pandas as pd
import ibm_db_dbi

In [3]:
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_hostname = "xxxxxxxxxxxxxxxxxxxxxxxxxxx"            # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_port = "50000"                    # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_uid = "xxxxx"                 # e.g. "abc12345"
dsn_pwd = "xxxxxxxx" 


In [4]:
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

conn = ibm_db.connect(dsn, "", "")
print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)


Connected to database:  BLUDB as user:  rbd60127 on host:  dashdb-txn-sbox-yp-lon02-01.services.eu-gb.bluemix.net


In [6]:
selectQuery = "select COLUMN_1, COLUMN_3, COLUMN_4, COLUMN_5 from ASSIGNDATA"
pconn = ibm_db_dbi.Connection(conn)
pdf = pd.read_sql(selectQuery, pconn)


In [7]:
pdf.shape

(193176, 4)

In [8]:
pdf[['COLUMN_3', 'COLUMN_4', 'COLUMN_5']] = pdf[['COLUMN_3', 'COLUMN_4', 'COLUMN_5']].apply(pd.to_numeric)

In [9]:

pdf.dtypes

COLUMN_1     object
COLUMN_3    float64
COLUMN_4    float64
COLUMN_5    float64
dtype: object

In [10]:
output = pdf["COLUMN_1"].astype("|S")
output.astype(str)
del pdf["COLUMN_1"]

In [11]:
pdf["output"] = output

In [12]:
pdf.dtypes

COLUMN_3    float64
COLUMN_4    float64
COLUMN_5    float64
output          |S1
dtype: object

In [14]:
from sklearn import preprocessing
y = pdf["output"]
X = pdf[['COLUMN_3', 'COLUMN_4', 'COLUMN_5']]
X= preprocessing.StandardScaler().fit(X).transform(X)
y_train = pd.get_dummies(y)
X[:5]

array([[0.71409338, 0.37529828, 0.0472978 ],
       [0.75759526, 0.31082772, 0.04682746],
       [0.96899235, 0.45342792, 0.14230604],
       [1.20627475, 0.51188854, 0.21379741],
       [1.62727136, 0.41190448, 0.2490727 ]])

In [15]:
pdf.describe()

,COLUMN_3,COLUMN_4,COLUMN_5
count,193176.000000,193176.000000,193176.000000
mean,1.068569,-5.238736,2.279133
std,6.659477,4.382100,5.090401
min,-19.788227,-19.656546,-19.680937
25%,-3.246575,-7.767239,-0.860716
50%,-0.490512,-5.459229,2.600552
75%,7.337181,-2.887557,6.249465
max,19.660885,19.858856,19.650410


In [16]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
import numpy as np

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1)

In [20]:
#from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
y = np.array(y_train)
#clf = RandomForestClassifier(max_depth=75, random_state=0, min_samples_split=7)
kvalues = [2, 4, 7, 10, 20, 50, 100]
maxacc=0
bestK = 1
for i in kvalues:
    neigh = KNeighborsClassifier(n_neighbors = i).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    acc = metrics.accuracy_score(y_test, yhat)
    if acc>maxacc:
        maxacc = acc
        bestK = i
print("Best K = {0}, Max Accuracy = {1}".format(bestK, maxacc))

Best K = 50, Max Accuracy = 0.6302929909928564


In [34]:
y_pred = clf.predict(X_test)
print(accuracy_score(y_pred, y_test))

0.3299226066746949
